# Introduction
I have utilized my skills in SQL, Python and Jupyter Notebook to perform different queries in a fast food business's database. Throughout this task, I have executed queries using Python code, interacted with an online server to insert values and update tables, and leveraged pandas to transform the query results into DataFrames. Finally, I stored these DataFrames in an Excel sheet to facilitate further exploration.

# Project Overview
The project revolves around a SQL database specifically designed for a fast food business. It encompasses several tables capturing essential aspects of the business operations. Here's an overview of the tables involved:

1. `Producto`
(Product): Contains a list of items in the menu, including their unique codes and prices.

2. `Empleado`
(Employee): Provides information about all employees, including their IDs, full names, social security numbers, shifts, and salaries.

3. `Repartidor`
(Delivery Person): Stores details about the delivery personnel, such as their IDs, full names, shifts, and incentives.

4. `Pedido`
(Order): Records various aspects of each order, including the order number, date, price, IDs of employees involved in ticketing, order preparation, and delivery, as well as the respective timestamps.

5. `esta_compuesto`
(Composed of): Establishes relationships between menu items by linking the codes of items that are composed of other items.

6. `consta`
(Constancy): Stores information about the items included in each order, specifying the item code, order number, and quantity requested.

7. `Pedidos_finalizados`
(Order Finished): Acts as a repository for completed orders, mirroring the structure of the PEDIDO table, and including an additional column for the elapsed time.

8. `Ranking_productos`
(Ranking of Products): Tracks the popularity of menu items by aggregating data from the consta table, including the item code, name, and the total number of times each item has been ordered.

# Instalo MYSQL CONNECTOR

In [2]:
!pip install mysql.connector

     --------------------------------------- 11.9/11.9 MB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for mysql.connector: started
  Running setup.py install for mysql.connector: finished with status 'done'


  DEPRECATION: mysql.connector is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importo Librerías

In [2]:
import mysql.connector
from mysql.connector import Error

In [3]:
import pandas as pd
from pandas import ExcelWriter

# Datos del Servidor

nombredb =  </br>
nombreusuario =  </br>
passw =  </br>
ip =  </br>


# Test de conexión

In [4]:
try:
    connection = mysql.connector.connect(host='34.175.251.45', #ip
                                         database='dbegg8vatjbot9', #name db
                                         user='u7xcmnsjmtujl',  #user name
                                         password='upgradehub' , connection_timeout=180)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print('Connected to MySQL database... MySQL Server version on', db_Info)

        cursor= connection.cursor()
        connection.commit()

except Error as e:
    print('Error while connecting to MySQL', e)
finally:
    #closing databes connection.
    if connection.is_connected():
        cursor.close()
        connection.close()
        print('MySQL connection is closed')

Connected to MySQL database... MySQL Server version on 5.7.39-42-log
MySQL connection is closed


# Conecto con el Servidor

In [5]:
connection = mysql.connector.connect(host='34.175.251.45',
                                         database='dbegg8vatjbot9',
                                         user='u7xcmnsjmtujl',
                                         password='upgradehub')
cursor= connection.cursor()

# Creo Tabla Producto

In [6]:
queryT1=""" 
CREATE TABLE IF NOT EXISTS PRODUCTO (
Codigo char(3) PRIMARY KEY,
Nombre VARCHAR(50) UNIQUE,
Precio FLOAT(6,2)); 
"""
cursor.execute(queryT1)

# Creo Tabla Empleado

In [7]:
queryT2=""" 
CREATE TABLE IF NOT EXISTS  EMPLEADO (
DNI CHAR(9) PRIMARY KEY,
Nombre VARCHAR(50),
Nss VARCHAR(11),
Turno enum('mañana', 'tarde','noche'),
Salario FLOAT(6,2));
"""
cursor.execute(queryT2)

# Creo Tabla Repartidor

In [8]:
queryT3=""" 
CREATE TABLE IF NOT EXISTS REPARTIDOR (
DNI CHAR(9) PRIMARY KEY,
Nombre VARCHAR(50),
Turno enum('mañana', 'tarde','noche'),
Incentivo FLOAT(6,2));
"""
cursor.execute(queryT3)

# Creo Tabla Pedido

In [9]:
queryT4=""" 
CREATE TABLE IF NOT EXISTS PEDIDO (
Numero char(4) PRIMARY KEY,
Fecha DATE NOT NULL,
Importe FLOAT(6,2),
DNI_ETM CHAR(9),
DNI_EP CHAR(9),
DNI_R CHAR(9),
Hora_tm time,
Hora_pre time,
Hora_rep time,
CONSTRAINT  Pe_dniETM_FK FOREIGN KEY (DNI_ETM) REFERENCES EMPLEADO(DNI) ON DELETE CASCADE,
CONSTRAINT  Pe_dniEP_FK FOREIGN KEY (DNI_EP) REFERENCES EMPLEADO(DNI) ON DELETE CASCADE,
CONSTRAINT  Pe_dniR_FK FOREIGN KEY (DNI_R) REFERENCES REPARTIDOR(DNI) ON DELETE CASCADE);
"""
cursor.execute(queryT4)

# Creo Tabla Está Compuesto

In [10]:
queryT5=""" 
CREATE TABLE IF NOT EXISTS esta_compuesto (
Codigo_P char(3),
Codigo_P_compuesto char(3),
CONSTRAINT EC_PK PRIMARY KEY (Codigo_P,Codigo_P_compuesto),
CONSTRAINT  EC_coP_FK FOREIGN KEY (Codigo_P) REFERENCES PRODUCTO(Codigo) ON DELETE CASCADE,
CONSTRAINT  EC_coPC_FK FOREIGN KEY (Codigo_P_compuesto) REFERENCES PRODUCTO(Codigo) ON DELETE CASCADE);
"""
cursor.execute(queryT5)

# Creo Tabla Consta

In [11]:
queryT6=""" 
CREATE TABLE IF NOT EXISTS consta(
Codigo_Pr char(3),
Numero_P char(4),
cantidad TINYINT,
CONSTRAINT CO_PK PRIMARY KEY (Codigo_Pr,Numero_P),
CONSTRAINT  CO_coP_FK FOREIGN KEY (Codigo_Pr) REFERENCES PRODUCTO(Codigo) ON DELETE CASCADE,
CONSTRAINT  CO_nuP_FK FOREIGN KEY (Numero_P) REFERENCES PEDIDO(Numero) ON DELETE CASCADE);
"""
cursor.execute(queryT6)

# Creo Tabla Pedidos Finalizados

In [12]:
queryT7=""" 
CREATE TABLE IF NOT EXISTS PEDIDOS_FINALIZADOS (
Numero char(4) PRIMARY KEY,
Fecha DATE NOT NULL,
Importe FLOAT(6,2),
DNI_ETM CHAR(9),
DNI_EP CHAR(9),
DNI_R CHAR(9),
Hora_tm time,
Hora_pre time,
Hora_rep time,
CONSTRAINT  Pe_dniETM1_FK FOREIGN KEY (DNI_ETM) REFERENCES EMPLEADO(DNI) ON DELETE CASCADE,
CONSTRAINT  Pe_dniEP1_FK FOREIGN KEY (DNI_EP) REFERENCES EMPLEADO(DNI) ON DELETE CASCADE,
CONSTRAINT  Pe_dniR1_FK FOREIGN KEY (DNI_R) REFERENCES REPARTIDOR(DNI) ON DELETE CASCADE,
Tiempo_transcurrido time);
"""
cursor.execute(queryT7)

# Creo Tabla Ranking Productos

In [13]:
queryT8=""" 
CREATE TABLE RANKING_PRODUCTOS(
Codigo char(3) PRIMARY KEY,
Nombre VARCHAR(50) UNIQUE,
Total int(3));
"""
cursor.execute(queryT8)

# Inserto Valores en Producto

In [14]:
queryT9=""" 
INSERT INTO PRODUCTO VALUES
('01','Hamburguesa',2.6),
('02','Patatas',2.0),
('03','tomate',0.50),
('04','Queso',1.0),
('05','Lechuga',0.50),
('06','Pollo',3.6),
('08','Bacon',1.5),
('09','Coca cola',3.0),
('19','Fanta',3.0),
('10','Nestea',3.0),
('18','Agua',2.0),
('11','Menú de Pollo',5.0),
('12','Menú de Hamburguesa con queso',6.0),
('13','Menú de Pollo con queso',6.0),
('14','Menú de Hamburguesa',6.0),
('15','Helado',1.0),
('16','Tarta',2.0),
('17','Fruta',1.0);
"""
cursor.execute(queryT9)
#Commit the changes to the database
connection.commit()

# Inserto Valores en Está Compuesto

In [15]:
queryT10 = """ 
INSERT INTO esta_compuesto VALUES 
('11','06'),
('11','02'),
('11','03'),
('11','18'),
('11','17'),
('12','01'),
('12','02'),
('12','04'),
('12','10'),
('12','16'),
('13','06'),
('13','08'),
('13','04'),
('13','02'),
('13','09'),
('13','15'),
('14','03'),
('14','05'),
('14','08'),
('14','01'),
('14','02'),
('14','09'),
('14','15');
"""
cursor.execute(queryT10)
connection.commit()


# Inserto Valores en Empleado

In [16]:
queryT11 = """ 
INSERT INTO EMPLEADO VALUES
('11111111Q','Luis Ramírez Pardo','23445556666' , 'mañana', 900),
('22222222S','María Sánchez Cid','11112223334' , 'tarde', 1000),
('33333333M','Martín Guerrero López','33344455566' , 'tarde', 1000),
('04444444T','Úrsula Delta Camacho', '11177788899', 'mañana', 900),
('55555555J','Carmen Hernández Pío','99966633311' , 'mañana', 900),
('77777777M','Pedro Jiménez Ruiz','23456785432' , 'tarde', 1000),
('99999999X','Raúl Rodrigo Roca','55566677789' , 'tarde', 1000),
('88888888O','Soledad Campillo Molina','00088877754' , 'noche', 1200),
('03232323P','María Luisa Galdón Ter','43534534567', 'noche', 1200),
('14567555L','Piedad Colmenero Zapillo','23456734534' , 'noche', 1200),
('14111155T','Sergio Lérida Campos','55577700089' , 'tarde', 1000);
"""
cursor.execute(queryT11)
connection.commit()


# Inserto Valores en Repartidor

In [17]:
queryT12 = """ 
INSERT INTO REPARTIDOR VALUES
('14188151T','Carlos Sánchez Ruíz', 'tarde', 300),
('11245621Q','Juan Pardo Rubio', 'noche', 400),
('04477744T','Laura Jiménez Jiménez', 'noche', 400),
('51235555J','Carmen Capel Pío', 'tarde', 300),
('55675675J','Juan Sánchez López', 'mañana', 200),
('99009900J','Alejandro Pardo López', 'mañana', 400);
"""
cursor.execute(queryT12)
connection.commit()

# Inserto Valores en Pedido

In [18]:
queryT13 = """ 
INSERT INTO PEDIDO VALUES
('0001','2020-10-15',10,'11111111Q','04444444T','55675675J','12:00','12:15','12:45'),
('0002','2020-11-11',15,'22222222S','77777777M','99009900J','13:30','13:45','14:05'),
('0003','2020-10-15',13,'77777777M','22222222S','99009900J','15:00','15:15','15:45'),
('0004','2020-11-10',13,'99999999X','77777777M','04477744T','14:02','14:30','14:45'),
('0005','2020-09-05',14,'14567555L','03232323P','14188151T','19:02','19:30','19:45'),
('0006','2020-11-15',23,'14567555L','88888888O','04477744T','21:02','21:35','21:45'),
('0007','2020-09-25',17,'03232323P','88888888O','14188151T','23:05','23:12','23:35'),
('0008','2020-09-15',14,'99999999X','33333333M','11245621Q','18:02','18:30','18:45'),
('0009','2020-11-23',25,'04444444T','55555555J','99009900J','11:02','11:30','11:55'),
('0010','2020-11-05',45,'88888888O','14567555L',null,'22:05','22:12',null),
('0011','2020-11-05',45,'88888888O','14567555L',null,'22:05',null,null);
"""
cursor.execute(queryT13)
connection.commit()

# Inserto Valores en Consta

In [19]:
queryT14 = """ 
INSERT INTO consta VALUES
('11','0001',2),
('12','0001',1),
('11','0002',1),
('12','0002',2),
('15','0002',2),
('14','0003',2),
('17','0003',1),
('13','0004',3),
('04','0004',1),
('13','0005',2),
('12','0005',1),
('02','0005',1),
('12','0006',2),
('13','0006',1),
('19','0006',1),
('18','0006',1),
('12','0007',2),
('09','0007',1),
('02','0007',1),
('13','0008',2),
('11','0008',1),
('13','0009',4),
('15','0009',1),
('12','0010',3),
('13','0010',3),
('16','0010',3),
('04','0010',3),
('04','0011',2);
"""
cursor.execute(queryT14)
connection.commit()

# Peticiones

### Obtener el listado de todos los productos

In [20]:
query1=""" SELECT * FROM PRODUCTO; """
cursor.execute(query1)
cursor.fetchall()

[('01', 'Hamburguesa', 2.6),
 ('02', 'Patatas', 2.0),
 ('03', 'tomate', 0.5),
 ('04', 'Queso', 1.0),
 ('05', 'Lechuga', 0.5),
 ('06', 'Pollo', 3.6),
 ('08', 'Bacon', 1.5),
 ('09', 'Coca cola', 3.0),
 ('10', 'Nestea', 3.0),
 ('11', 'Menú de Pollo', 5.0),
 ('12', 'Menú de Hamburguesa con queso', 6.0),
 ('13', 'Menú de Pollo con queso', 6.0),
 ('14', 'Menú de Hamburguesa', 6.0),
 ('15', 'Helado', 1.0),
 ('16', 'Tarta', 2.0),
 ('17', 'Fruta', 1.0),
 ('18', 'Agua', 2.0),
 ('19', 'Fanta', 3.0)]

In [21]:
df1=pd.read_sql(query1, con=connection)
df1

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2674531143.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1=pd.read_sql(query1, con=connection)


,Codigo,Nombre,Precio
0,01,Hamburguesa,2.6
1,02,Patatas,2.0
2,03,tomate,0.5
3,04,Queso,1.0
4,05,Lechuga,0.5
5,06,Pollo,3.6
6,08,Bacon,1.5
7,09,Coca cola,3.0
8,10,Nestea,3.0
9,11,Menú de Pollo,5.0


### Obtener el número de pedido, dni del repartidor y hora de reparto de aquellos pedidos que se toman nota después de las siete de la tarde.

In [22]:
query2=""" SELECT
	Numero,
    DNI_R,
    Hora_rep
FROM 
	PEDIDO
WHERE
	(Hora_tm>'19:00:00') AND (Hora_rep is not null); """
cursor.execute(query2)
cursor.fetchall()

[('0005', '14188151T', datetime.timedelta(seconds=71100)),
 ('0006', '04477744T', datetime.timedelta(seconds=78300)),
 ('0007', '14188151T', datetime.timedelta(seconds=84900))]

In [23]:
df2=pd.read_sql(query2, con=connection)
df2

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\3218547425.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2=pd.read_sql(query2, con=connection)


,Numero,DNI_R,Hora_rep
0,0005,14188151T,0 days 19:45:00
1,0006,04477744T,0 days 21:45:00
2,0007,14188151T,0 days 23:35:00


### Obtener todos los campos de empleados que cobran entre 900 y 1000 euros.

In [24]:
query3=""" SELECT *
FROM
	EMPLEADO
WHERE 
	Salario BETWEEN 900 AND 1000; """
cursor.execute(query3)
cursor.fetchall()

[('04444444T', 'Úrsula Delta Camacho', '11177788899', 'mañana', 900.0),
 ('11111111Q', 'Luis Ramírez Pardo', '23445556666', 'mañana', 900.0),
 ('14111155T', 'Sergio Lérida Campos', '55577700089', 'tarde', 1000.0),
 ('22222222S', 'María Sánchez Cid', '11112223334', 'tarde', 1000.0),
 ('33333333M', 'Martín Guerrero López', '33344455566', 'tarde', 1000.0),
 ('55555555J', 'Carmen Hernández Pío', '99966633311', 'mañana', 900.0),
 ('77777777M', 'Pedro Jiménez Ruiz', '23456785432', 'tarde', 1000.0),
 ('99999999X', 'Raúl Rodrigo Roca', '55566677789', 'tarde', 1000.0)]

In [25]:
df3=pd.read_sql(query3, con=connection)
df3

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1316144687.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3=pd.read_sql(query3, con=connection)


,DNI,Nombre,Nss,Turno,Salario
0,04444444T,Úrsula Delta Camacho,11177788899,mañana,900.0
1,11111111Q,Luis Ramírez Pardo,23445556666,mañana,900.0
2,14111155T,Sergio Lérida Campos,55577700089,tarde,1000.0
3,22222222S,María Sánchez Cid,11112223334,tarde,1000.0
4,33333333M,Martín Guerrero López,33344455566,tarde,1000.0
5,55555555J,Carmen Hernández Pío,99966633311,mañana,900.0
6,77777777M,Pedro Jiménez Ruiz,23456785432,tarde,1000.0
7,99999999X,Raúl Rodrigo Roca,55566677789,tarde,1000.0


### Obtener el número de pedido e importe de aquellos que han sido registrados en el mes de noviembre de 2020 y su importe es mayor a 15 Euros.

In [26]:
query4=""" SELECT
	Numero,
    Importe
FROM 
	PEDIDO
WHERE
	(Fecha like '2020-11-%') AND (Importe >15); """
cursor.execute(query4)
cursor.fetchall()

[('0006', 23.0), ('0009', 25.0), ('0010', 45.0), ('0011', 45.0)]

In [27]:
df4=pd.read_sql(query4, con=connection)
df4

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2846340848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4=pd.read_sql(query4, con=connection)


,Numero,Importe
0,0006,23.0
1,0009,25.0
2,0010,45.0
3,0011,45.0


### Obtener por cada uno de los repartidores su DNI junto a la cantidad de pedidos entregados.

In [28]:
query5=""" SELECT
	r.DNI,
    COUNT(p.Numero) CantidadPedidos
FROM REPARTIDOR r
INNER JOIN
	PEDIDO p ON r.DNI=p.DNI_R
GROUP BY
	r.DNI; """
cursor.execute(query5)
cursor.fetchall()

[('04477744T', 2),
 ('11245621Q', 1),
 ('14188151T', 2),
 ('55675675J', 1),
 ('99009900J', 3)]

In [29]:
df5=pd.read_sql(query5, con=connection)
df5

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\865239477.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5=pd.read_sql(query5, con=connection)


,DNI,CantidadPedidos
0,04477744T,2
1,11245621Q,1
2,14188151T,2
3,55675675J,1
4,99009900J,3


### Obtener por cada mes (con el formato de nombre y no de número ej. Noviembre),la cantidad de pedidos realizados.

In [30]:
query6=""" SELECT
	MONTHNAME(Fecha) Mes,
    COUNT(Numero) Pedidos
FROM
	PEDIDO
WHERE 
	(Numero AND Fecha) is not null
GROUP BY
	MONTHNAME(Fecha)
ORDER BY
	Mes DESC; """
cursor.execute(query6)
cursor.fetchall()

[('September', 3), ('October', 2), ('November', 6)]

In [31]:
df6=pd.read_sql(query6, con=connection)
df6

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\3169886837.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6=pd.read_sql(query6, con=connection)


,Mes,Pedidos
0,September,3
1,October,2
2,November,6


### Obtener un listado con el nombre y dni de los empleados en el mismo campo y anteponiendo el dni al nombre de la siguiente forma (ej; 45776633P, Juan Rodríguez López) que tenga turno de “tarde” o "noche". Ordenar el listado por dni.

In [32]:
query7=""" SELECT
	CONCAT(DNI, ', ', Nombre) Empleado,
    Turno
FROM 
	EMPLEADO
WHERE
	Turno in ('tarde','noche')
ORDER BY
	DNI ASC; """
cursor.execute(query7)
cursor.fetchall()

[('03232323P, María Luisa Galdón Ter', 'noche'),
 ('14111155T, Sergio Lérida Campos', 'tarde'),
 ('14567555L, Piedad Colmenero Zapillo', 'noche'),
 ('22222222S, María Sánchez Cid', 'tarde'),
 ('33333333M, Martín Guerrero López', 'tarde'),
 ('77777777M, Pedro Jiménez Ruiz', 'tarde'),
 ('88888888O, Soledad Campillo Molina', 'noche'),
 ('99999999X, Raúl Rodrigo Roca', 'tarde')]

In [33]:
df7=pd.read_sql(query7, con=connection)
df7

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2296241394.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df7=pd.read_sql(query7, con=connection)


,Empleado,Turno
0,"03232323P, María Luisa Galdón Ter",noche
1,"14111155T, Sergio Lérida Campos",tarde
2,"14567555L, Piedad Colmenero Zapillo",noche
3,"22222222S, María Sánchez Cid",tarde
4,"33333333M, Martín Guerrero López",tarde
5,"77777777M, Pedro Jiménez Ruiz",tarde
6,"88888888O, Soledad Campillo Molina",noche
7,"99999999X, Raúl Rodrigo Roca",tarde


### Obtener el nombre, código y precio de aquellos productos que superen o igualen la media de todos los precios. Ordenar de mayor a menor precio.

In [34]:
query8=""" SELECT
	Nombre,
    Codigo,
    Precio
FROM
	PRODUCTO
WHERE
	(Precio>=(SELECT AVG(Precio) FROM PRODUCTO))
ORDER BY
	Precio DESC; """
cursor.execute(query8)
cursor.fetchall()

[('Menú de Hamburguesa con queso', '12', 6.0),
 ('Menú de Pollo con queso', '13', 6.0),
 ('Menú de Hamburguesa', '14', 6.0),
 ('Menú de Pollo', '11', 5.0),
 ('Pollo', '06', 3.6),
 ('Coca cola', '09', 3.0),
 ('Nestea', '10', 3.0),
 ('Fanta', '19', 3.0)]

In [35]:
df8=pd.read_sql(query8, con=connection)
df8

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\52602469.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df8=pd.read_sql(query8, con=connection)


,Nombre,Codigo,Precio
0,Menú de Hamburguesa con queso,12,6.0
1,Menú de Pollo con queso,13,6.0
2,Menú de Hamburguesa,14,6.0
3,Menú de Pollo,11,5.0
4,Pollo,06,3.6
5,Coca cola,09,3.0
6,Nestea,10,3.0
7,Fanta,19,3.0


### Obtener un listado con el nombre y DNI de los empleados que no han preparado nunca ningún pedido.

In [36]:
query9=""" SELECT
	emp.DNI,
    emp.Nombre
FROM EMPLEADO emp
LEFT OUTER JOIN
	PEDIDO p ON emp.DNI=p.DNI_EP
WHERE
	p.DNI_EP is null; """
cursor.execute(query9)
cursor.fetchall()

[('11111111Q', 'Luis Ramírez Pardo'),
 ('14111155T', 'Sergio Lérida Campos'),
 ('99999999X', 'Raúl Rodrigo Roca')]

In [37]:
df9=pd.read_sql(query9, con=connection)
df9

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1876838131.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df9=pd.read_sql(query9, con=connection)


,DNI,Nombre
0,11111111Q,Luis Ramírez Pardo
1,14111155T,Sergio Lérida Campos
2,99999999X,Raúl Rodrigo Roca


### Obtener el código, nombre y precio de los productos(estos dos últimos en el mismo campo) que están contenidos en los pedidos que ha tomado nota "Luis" o "María Luisa". Ordena el listado de mayor a menor valor por fecha del pedido.

In [38]:
query10=""" SELECT 
	pro.Codigo,
    CONCAT(pro.Nombre, ', ', pro.Precio) NombrePrecio
FROM
	consta con
INNER JOIN
	PEDIDO p ON con.Numero_P=p.Numero
INNER JOIN
	PRODUCTO pro ON con.Codigo_Pr=pro.Codigo
WHERE p.DNI_ETM IN (
SELECT
    emp.DNI
FROM 
	EMPLEADO emp
INNER JOIN
	PEDIDO p ON emp.DNI=p.DNI_ETM
WHERE
    (emp.Nombre like ('Luís%')) or (emp.Nombre like ('María Luisa%'))
    )
ORDER BY
	p.Fecha; """
cursor.execute(query10)
cursor.fetchall()

[('02', 'Patatas, 2.00'),
 ('09', 'Coca cola, 3.00'),
 ('12', 'Menú de Hamburguesa con queso, 6.00'),
 ('11', 'Menú de Pollo, 5.00'),
 ('12', 'Menú de Hamburguesa con queso, 6.00')]

In [39]:
df10=pd.read_sql(query10, con=connection)
df10

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1228663485.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df10=pd.read_sql(query10, con=connection)


,Codigo,NombrePrecio
0,02,"Patatas, 2.00"
1,09,"Coca cola, 3.00"
2,12,"Menú de Hamburguesa con queso, 6.00"
3,11,"Menú de Pollo, 5.00"
4,12,"Menú de Hamburguesa con queso, 6.00"


### Obtener por cada repartidor, su nombre, cantidad de pedidos y el tiempo medio que tardan en entregar los pedidos una vez preparados. Ordenar el listado el tiempo medio que tardan en entregarlos.

In [40]:
query11=""" SELECT
	re.Nombre,
    COUNT(p.Numero) TotalPedidos,
	AVG(TIMESTAMPDIFF(MINUTE, p.Hora_pre, p.Hora_rep)) MediaTiempoEntrega
FROM
	PEDIDO p
INNER JOIN
	REPARTIDOR re ON re.DNI=p.DNI_R
GROUP BY
	re.Nombre
ORDER BY
	AVG(TIMESTAMPDIFF(MINUTE, p.Hora_pre, p.Hora_rep)); """
cursor.execute(query11)
cursor.fetchall()

[('Laura Jiménez Jiménez', 2, Decimal('12.5000')),
 ('Juan Pardo Rubio', 1, Decimal('15.0000')),
 ('Carlos Sánchez Ruíz', 2, Decimal('19.0000')),
 ('Alejandro Pardo López', 3, Decimal('25.0000')),
 ('Juan Sánchez López', 1, Decimal('30.0000'))]

In [41]:
df11=pd.read_sql(query11, con=connection)
df11

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2813535984.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df11=pd.read_sql(query11, con=connection)


,Nombre,TotalPedidos,MediaTiempoEntrega
0,Laura Jiménez Jiménez,2,12.5
1,Juan Pardo Rubio,1,15.0
2,Carlos Sánchez Ruíz,2,19.0
3,Alejandro Pardo López,3,25.0
4,Juan Sánchez López,1,30.0


### Obtener un listado obteniendo el código, nombre y el precio de los productos cuyo precios sea el más barato o el más caro de todos.

In [42]:
query12=""" SELECT *
FROM
	PRODUCTO
WHERE
	(Precio in (
    SELECT
		MAX(Precio)
	FROM 
		PRODUCTO))
	OR
    (Precio in (
    SELECT
        MIN(Precio)
	FROM
		PRODUCTO
        ))
	ORDER BY
		Precio; """
cursor.execute(query12)
cursor.fetchall()

[('03', 'tomate', 0.5),
 ('05', 'Lechuga', 0.5),
 ('12', 'Menú de Hamburguesa con queso', 6.0),
 ('13', 'Menú de Pollo con queso', 6.0),
 ('14', 'Menú de Hamburguesa', 6.0)]

In [43]:
df12=pd.read_sql(query12, con=connection)
df12

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2004618977.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df12=pd.read_sql(query12, con=connection)


,Codigo,Nombre,Precio
0,03,tomate,0.5
1,05,Lechuga,0.5
2,12,Menú de Hamburguesa con queso,6.0
3,13,Menú de Pollo con queso,6.0
4,14,Menú de Hamburguesa,6.0


### Obtener por cada producto , el nombre y código el número total de pedidos en los que se encuentra teniendo en cuenta que el total de pedidos en los cuales se encuentre sea superior o igual a dos. Ordena el listado de mayor a menor número de productos.

In [44]:
query13=""" SELECT 
	pro.Codigo,
    pro.Nombre,
    SUM(con.cantidad) TotalPedidos
FROM
	consta con
INNER JOIN
	PEDIDO p ON con.Numero_P=p.Numero
INNER JOIN
	PRODUCTO pro ON con.Codigo_Pr=pro.Codigo
WHERE (
SELECT SUM(cantidad)
FROM consta
WHERE Codigo_Pr= pro.Codigo)>=2
GROUP BY 
	pro.Codigo
ORDER BY
	SUM(con.cantidad) DESC; """
cursor.execute(query13)
cursor.fetchall()

[('13', 'Menú de Pollo con queso', Decimal('15')),
 ('12', 'Menú de Hamburguesa con queso', Decimal('11')),
 ('04', 'Queso', Decimal('6')),
 ('11', 'Menú de Pollo', Decimal('4')),
 ('15', 'Helado', Decimal('3')),
 ('16', 'Tarta', Decimal('3')),
 ('02', 'Patatas', Decimal('2')),
 ('14', 'Menú de Hamburguesa', Decimal('2'))]

In [45]:
df13=pd.read_sql(query13, con=connection)
df13

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1720338336.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df13=pd.read_sql(query13, con=connection)


,Codigo,Nombre,TotalPedidos
0,13,Menú de Pollo con queso,15.0
1,12,Menú de Hamburguesa con queso,11.0
2,04,Queso,6.0
3,11,Menú de Pollo,4.0
4,15,Helado,3.0
5,16,Tarta,3.0
6,02,Patatas,2.0
7,14,Menú de Hamburguesa,2.0


### Mostrar listado de los empleados (código y NSS en la misma columna) que han tomado nota de algún pedido y contienen el producto de código 13 y además el repartidor sea 'Laura'.

In [46]:
query14=""" SELECT 
	CONCAT(emp.DNI, ', ',emp.NSS) Identificación,
    emp.Nombre
FROM 
	PEDIDO p
RIGHT OUTER JOIN 
	EMPLEADO emp ON emp.DNI = p.DNI_ETM
RIGHT OUTER JOIN 
	REPARTIDOR r ON r.DNI = p.DNI_R
INNER JOIN 
	consta con ON con.Numero_P = p.Numero
INNER JOIN 
	PRODUCTO pro ON con.Codigo_Pr = pro.Codigo
WHERE 
	(r.Nombre LIKE 'Laura%') AND (p.DNI_ETM IS NOT NULL) AND (pro.Codigo = '13'); """
cursor.execute(query14)
cursor.fetchall()

[('99999999X, 55566677789', 'Raúl Rodrigo Roca'),
 ('14567555L, 23456734534', 'Piedad Colmenero Zapillo')]

In [47]:
df14=pd.read_sql(query14, con=connection)
df14

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2495470260.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df14=pd.read_sql(query14, con=connection)


,Identificación,Nombre
0,"99999999X, 55566677789",Raúl Rodrigo Roca
1,"14567555L, 23456734534",Piedad Colmenero Zapillo


### Obtener el nombre del producto que es menú junto con el código de los productos que lo componen en aquellos pedidos del mes de septiembre de 2020.

In [48]:
query15=""" SELECT
	pro.Nombre,
    comp.Codigo_P_compuesto
FROM 
	PRODUCTO pro
INNER JOIN
	esta_compuesto comp ON pro.Codigo=comp.Codigo_P
INNER JOIN
	consta con ON con.Codigo_Pr = pro.Codigo
INNER JOIN
	PEDIDO p ON con.Numero_P = p.Numero
WHERE (pro.Codigo in (comp.Codigo_P)) AND (p.Fecha like '2020-09-%'); """
cursor.execute(query15)
cursor.fetchall()

[('Menú de Hamburguesa con queso', '01'),
 ('Menú de Hamburguesa con queso', '02'),
 ('Menú de Hamburguesa con queso', '04'),
 ('Menú de Hamburguesa con queso', '10'),
 ('Menú de Hamburguesa con queso', '16'),
 ('Menú de Pollo con queso', '02'),
 ('Menú de Pollo con queso', '04'),
 ('Menú de Pollo con queso', '06'),
 ('Menú de Pollo con queso', '08'),
 ('Menú de Pollo con queso', '09'),
 ('Menú de Pollo con queso', '15'),
 ('Menú de Hamburguesa con queso', '01'),
 ('Menú de Hamburguesa con queso', '02'),
 ('Menú de Hamburguesa con queso', '04'),
 ('Menú de Hamburguesa con queso', '10'),
 ('Menú de Hamburguesa con queso', '16'),
 ('Menú de Pollo', '02'),
 ('Menú de Pollo', '03'),
 ('Menú de Pollo', '06'),
 ('Menú de Pollo', '17'),
 ('Menú de Pollo', '18'),
 ('Menú de Pollo con queso', '02'),
 ('Menú de Pollo con queso', '04'),
 ('Menú de Pollo con queso', '06'),
 ('Menú de Pollo con queso', '08'),
 ('Menú de Pollo con queso', '09'),
 ('Menú de Pollo con queso', '15')]

In [49]:
df15=pd.read_sql(query15, con=connection)
df15

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\703776082.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df15=pd.read_sql(query15, con=connection)


,Nombre,Codigo_P_compuesto
0,Menú de Hamburguesa con queso,01
1,Menú de Hamburguesa con queso,02
2,Menú de Hamburguesa con queso,04
3,Menú de Hamburguesa con queso,10
4,Menú de Hamburguesa con queso,16
5,Menú de Pollo con queso,02
6,Menú de Pollo con queso,04
7,Menú de Pollo con queso,06
8,Menú de Pollo con queso,08
9,Menú de Pollo con queso,09


### Insertar un nuevo registro en la tabla EMPLEADOS.

In [50]:
query16=""" INSERT INTO EMPLEADO (DNI, Nombre, Nss, Turno, Salario)
VALUES ('98567432x', 'Luis Fernández Cáceres', '09809809822', 'tarde', 850.00); """
cursor.execute(query16)
connection.commit()

In [51]:
query16_1=""" SELECT *
FROM 
	EMPLEADO
WHERE
	DNI='98567432x' """
cursor.execute(query16_1)
cursor.fetchall()

[('98567432x', 'Luis Fernández Cáceres', '09809809822', 'tarde', 850.0)]

In [52]:
df16=pd.read_sql(query16_1, con=connection)
df16

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\4142588565.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df16=pd.read_sql(query16_1, con=connection)


,DNI,Nombre,Nss,Turno,Salario
0,98567432x,Luis Fernández Cáceres,09809809822,tarde,850.0


### El pedido con número 010 se la ha asignado el repartidor 11245621Q y se ha sido entregado a la hora 22:20:45.

In [53]:
query17=""" UPDATE PEDIDO
SET 
	DNI_R = '11245621Q', Hora_rep = '22:20:45'
WHERE 
	Numero = '0010'; """
cursor.execute(query17)
connection.commit()

In [54]:
query17_1=""" SELECT *
FROM
    PEDIDO
WHERE
	Numero = '0010'; """
cursor.execute(query17_1)
cursor.fetchall()

[('0010',
  datetime.date(2020, 11, 5),
  45.0,
  '88888888O',
  '14567555L',
  '11245621Q',
  datetime.timedelta(seconds=79500),
  datetime.timedelta(seconds=79920),
  datetime.timedelta(seconds=80445))]

In [55]:
df17=pd.read_sql(query17_1, con=connection)
df17

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\2261152747.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df17=pd.read_sql(query17_1, con=connection)


,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep
0,0010,2020-11-05,45.0,88888888O,14567555L,11245621Q,0 days 22:05:00,0 days 22:12:00,0 days 22:20:45


### El producto fruta (17) está agotado, por favor borrar de la tabla correspondiente

In [56]:
query18=""" DELETE FROM PRODUCTO WHERE Codigo='17' """
cursor.execute(query18)
connection.commit()

In [57]:
query18_1=""" SELECT *
FROM
    PRODUCTO
"""
cursor.execute(query18_1)
cursor.fetchall()

[('01', 'Hamburguesa', 2.6),
 ('02', 'Patatas', 2.0),
 ('03', 'tomate', 0.5),
 ('04', 'Queso', 1.0),
 ('05', 'Lechuga', 0.5),
 ('06', 'Pollo', 3.6),
 ('08', 'Bacon', 1.5),
 ('09', 'Coca cola', 3.0),
 ('10', 'Nestea', 3.0),
 ('11', 'Menú de Pollo', 5.0),
 ('12', 'Menú de Hamburguesa con queso', 6.0),
 ('13', 'Menú de Pollo con queso', 6.0),
 ('14', 'Menú de Hamburguesa', 6.0),
 ('15', 'Helado', 1.0),
 ('16', 'Tarta', 2.0),
 ('18', 'Agua', 2.0),
 ('19', 'Fanta', 3.0)]

In [58]:
df18=pd.read_sql(query18_1, con=connection)
df18

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\88820627.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df18=pd.read_sql(query18_1, con=connection)


,Codigo,Nombre,Precio
0,01,Hamburguesa,2.6
1,02,Patatas,2.0
2,03,tomate,0.5
3,04,Queso,1.0
4,05,Lechuga,0.5
5,06,Pollo,3.6
6,08,Bacon,1.5
7,09,Coca cola,3.0
8,10,Nestea,3.0
9,11,Menú de Pollo,5.0


### Insertar los siguientes datos en la tabla PEDIDO teniendo en cuenta que debes insertar sólo los valores.

In [59]:
query19_1 = """
INSERT INTO PEDIDO (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep)
VALUES ('0014', '2021-01-15', '13', '99999999X', '55555555J', '04477744T', '21:20:00', '21:30:00', '21:51:00');
"""

query19_2 = """
INSERT INTO PEDIDO (Numero, Fecha, Importe, DNI_ETM, DNI_EP, Hora_tm, Hora_pre)
VALUES ('0012', '2020-11-16', '20', '03232323P', '04444444T', '18:00:00', '18:03:00');
"""

query19_3 = """
INSERT INTO PEDIDO (Numero, Fecha, Importe, DNI_ETM, Hora_tm)
VALUES ('0013', '2021-01-01', '67', '55555555J', '14:00:00');
"""

cursor.execute(query19_1)
cursor.execute(query19_2)
cursor.execute(query19_3)
connection.commit()


In [60]:
query19_4=""" 
SELECT *
FROM 
	PEDIDO
WHERE
	Numero in ('0012','0013','0014');
"""
cursor.execute(query19_4)
cursor.fetchall()

[('0012',
  datetime.date(2020, 11, 16),
  20.0,
  '03232323P',
  '04444444T',
  None,
  datetime.timedelta(seconds=64800),
  datetime.timedelta(seconds=64980),
  None),
 ('0013',
  datetime.date(2021, 1, 1),
  67.0,
  '55555555J',
  None,
  None,
  datetime.timedelta(seconds=50400),
  None,
  None),
 ('0014',
  datetime.date(2021, 1, 15),
  13.0,
  '99999999X',
  '55555555J',
  '04477744T',
  datetime.timedelta(seconds=76800),
  datetime.timedelta(seconds=77400),
  datetime.timedelta(seconds=78660))]

In [61]:
df19=pd.read_sql(query19_4, con=connection)
df19

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\205878715.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df19=pd.read_sql(query19_4, con=connection)


,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep
0,0012,2020-11-16,20.0,03232323P,04444444T,None,0 days 18:00:00,0 days 18:03:00,NaT
1,0013,2021-01-01,67.0,55555555J,None,None,0 days 14:00:00,NaT,NaT
2,0014,2021-01-15,13.0,99999999X,55555555J,04477744T,0 days 21:20:00,0 days 21:30:00,0 days 21:51:00


### Incrementar un 10% el importe de todos los pedidos que se han realizado en el mes de Noviembre de 2020.(Debes hacerlo con una única sentencia).

In [62]:
query20=""" 
UPDATE 
	PEDIDO
SET 
	Importe = Importe * 1.1
WHERE 
	Fecha like '2020-11-%'; 
"""
cursor.execute(query20)
connection.commit()

In [63]:
query20_1=""" 
SELECT *
FROM 
	PEDIDO
WHERE
	Fecha like '2020-11-%';
"""
cursor.execute(query20_1)
cursor.fetchall()

[('0002',
  datetime.date(2020, 11, 11),
  16.5,
  '22222222S',
  '77777777M',
  '99009900J',
  datetime.timedelta(seconds=48600),
  datetime.timedelta(seconds=49500),
  datetime.timedelta(seconds=50700)),
 ('0004',
  datetime.date(2020, 11, 10),
  14.3,
  '99999999X',
  '77777777M',
  '04477744T',
  datetime.timedelta(seconds=50520),
  datetime.timedelta(seconds=52200),
  datetime.timedelta(seconds=53100)),
 ('0006',
  datetime.date(2020, 11, 15),
  25.3,
  '14567555L',
  '88888888O',
  '04477744T',
  datetime.timedelta(seconds=75720),
  datetime.timedelta(seconds=77700),
  datetime.timedelta(seconds=78300)),
 ('0009',
  datetime.date(2020, 11, 23),
  27.5,
  '04444444T',
  '55555555J',
  '99009900J',
  datetime.timedelta(seconds=39720),
  datetime.timedelta(seconds=41400),
  datetime.timedelta(seconds=42900)),
 ('0010',
  datetime.date(2020, 11, 5),
  49.5,
  '88888888O',
  '14567555L',
  '11245621Q',
  datetime.timedelta(seconds=79500),
  datetime.timedelta(seconds=79920),
  datetim

In [64]:
df20=pd.read_sql(query20_1, con=connection)
df20

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\3557362551.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df20=pd.read_sql(query20_1, con=connection)


,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep
0,0002,2020-11-11,16.5,22222222S,77777777M,99009900J,0 days 13:30:00,0 days 13:45:00,0 days 14:05:00
1,0004,2020-11-10,14.3,99999999X,77777777M,04477744T,0 days 14:02:00,0 days 14:30:00,0 days 14:45:00
2,0006,2020-11-15,25.3,14567555L,88888888O,04477744T,0 days 21:02:00,0 days 21:35:00,0 days 21:45:00
3,0009,2020-11-23,27.5,04444444T,55555555J,99009900J,0 days 11:02:00,0 days 11:30:00,0 days 11:55:00
4,0010,2020-11-05,49.5,88888888O,14567555L,11245621Q,0 days 22:05:00,0 days 22:12:00,0 days 22:20:45
5,0011,2020-11-05,49.5,88888888O,14567555L,None,0 days 22:05:00,NaT,NaT
6,0012,2020-11-16,22.0,03232323P,04444444T,None,0 days 18:00:00,0 days 18:03:00,NaT


### Eliminar los pedidos cuyo tiempo de reparto ha sido de 25 minutos o más (desde que éstos estaban preparados) y además estaban asignados al repartidor de nombre Alejandro Pardo López. (Debes hacerlo con una única sentencia).

In [65]:
query21=""" 
DELETE FROM 
	PEDIDO
WHERE DNI_R IN (
    SELECT DNI
    FROM REPARTIDOR
    WHERE Nombre = 'Alejandro Pardo López'
)
AND ( TIMESTAMPDIFF(MINUTE, Hora_pre, Hora_rep)>=25);
"""
cursor.execute(query21)
connection.commit()

In [66]:
query21_1=""" 
SELECT *
FROM 
	PEDIDO
"""
cursor.execute(query21_1)
cursor.fetchall()

[('0001',
  datetime.date(2020, 10, 15),
  10.0,
  '11111111Q',
  '04444444T',
  '55675675J',
  datetime.timedelta(seconds=43200),
  datetime.timedelta(seconds=44100),
  datetime.timedelta(seconds=45900)),
 ('0002',
  datetime.date(2020, 11, 11),
  16.5,
  '22222222S',
  '77777777M',
  '99009900J',
  datetime.timedelta(seconds=48600),
  datetime.timedelta(seconds=49500),
  datetime.timedelta(seconds=50700)),
 ('0004',
  datetime.date(2020, 11, 10),
  14.3,
  '99999999X',
  '77777777M',
  '04477744T',
  datetime.timedelta(seconds=50520),
  datetime.timedelta(seconds=52200),
  datetime.timedelta(seconds=53100)),
 ('0005',
  datetime.date(2020, 9, 5),
  14.0,
  '14567555L',
  '03232323P',
  '14188151T',
  datetime.timedelta(seconds=68520),
  datetime.timedelta(seconds=70200),
  datetime.timedelta(seconds=71100)),
 ('0006',
  datetime.date(2020, 11, 15),
  25.3,
  '14567555L',
  '88888888O',
  '04477744T',
  datetime.timedelta(seconds=75720),
  datetime.timedelta(seconds=77700),
  datetime

In [67]:
df21=pd.read_sql(query21_1, con=connection)
df21

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1616728349.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df21=pd.read_sql(query21_1, con=connection)


,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep
0,0001,2020-10-15,10.0,11111111Q,04444444T,55675675J,0 days 12:00:00,0 days 12:15:00,0 days 12:45:00
1,0002,2020-11-11,16.5,22222222S,77777777M,99009900J,0 days 13:30:00,0 days 13:45:00,0 days 14:05:00
2,0004,2020-11-10,14.3,99999999X,77777777M,04477744T,0 days 14:02:00,0 days 14:30:00,0 days 14:45:00
3,0005,2020-09-05,14.0,14567555L,03232323P,14188151T,0 days 19:02:00,0 days 19:30:00,0 days 19:45:00
4,0006,2020-11-15,25.3,14567555L,88888888O,04477744T,0 days 21:02:00,0 days 21:35:00,0 days 21:45:00
5,0007,2020-09-25,17.0,03232323P,88888888O,14188151T,0 days 23:05:00,0 days 23:12:00,0 days 23:35:00
6,0008,2020-09-15,14.0,99999999X,33333333M,11245621Q,0 days 18:02:00,0 days 18:30:00,0 days 18:45:00
7,0010,2020-11-05,49.5,88888888O,14567555L,11245621Q,0 days 22:05:00,0 days 22:12:00,0 days 22:20:45
8,0011,2020-11-05,49.5,88888888O,14567555L,None,0 days 22:05:00,NaT,NaT
9,0012,2020-11-16,22.0,03232323P,04444444T,None,0 days 18:00:00,0 days 18:03:00,NaT


### Aumentar el incentivo de los repartidores en 50 euros, a todos aquellos que han repartido dos o más pedidos. (Debes hacerlo con una única sentencia).

In [68]:
query22=""" 
UPDATE 
	REPARTIDOR
SET 
	Incentivo = Incentivo + 50
WHERE DNI IN (
    SELECT DNI_R
    FROM PEDIDO
    GROUP BY DNI_R
    HAVING COUNT(*) >= 2
);
"""
cursor.execute(query22)
connection.commit()

In [69]:
query22_1=""" 
SELECT *
FROM 
	REPARTIDOR
"""
cursor.execute(query22_1)
cursor.fetchall()

[('04477744T', 'Laura Jiménez Jiménez', 'noche', 450.0),
 ('11245621Q', 'Juan Pardo Rubio', 'noche', 450.0),
 ('14188151T', 'Carlos Sánchez Ruíz', 'tarde', 350.0),
 ('51235555J', 'Carmen Capel Pío', 'tarde', 300.0),
 ('55675675J', 'Juan Sánchez López', 'mañana', 200.0),
 ('99009900J', 'Alejandro Pardo López', 'mañana', 400.0)]

In [70]:
df22=pd.read_sql(query22_1, con=connection)
df22

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\1293257701.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df22=pd.read_sql(query22_1, con=connection)


,DNI,Nombre,Turno,Incentivo
0,04477744T,Laura Jiménez Jiménez,noche,450.0
1,11245621Q,Juan Pardo Rubio,noche,450.0
2,14188151T,Carlos Sánchez Ruíz,tarde,350.0
3,51235555J,Carmen Capel Pío,tarde,300.0
4,55675675J,Juan Sánchez López,mañana,200.0
5,99009900J,Alejandro Pardo López,mañana,400.0


### Insertar todos los pedidos repartidos en la tabla PEDIDOS_FINALIZADOS incluyendo además de los campos propios de la tabla pedidos, el tiempo transcurrido desde que se toma nota hasta su reparto.(Debes hacerlo con una única sentencia)

In [71]:
query23=""" 
INSERT INTO 
	PEDIDOS_FINALIZADOS (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep, Tiempo_transcurrido)
SELECT 
	p.Numero, p.Fecha, p.Importe, p.DNI_ETM, p.DNI_EP, p.DNI_R, p.Hora_tm, p.Hora_pre, p.Hora_rep, TIMESTAMPDIFF(MINUTE, Hora_tm, Hora_rep) AS Tiempo_transcurrido
FROM 
	PEDIDO p
WHERE 
	(p.Hora_rep IS NOT NULL);
"""
cursor.execute(query23)
connection.commit()

In [72]:
query23_1=""" 
SELECT *
FROM
	PEDIDOS_FINALIZADOS
"""
cursor.execute(query23_1)
cursor.fetchall()

[('0001',
  datetime.date(2020, 10, 15),
  10.0,
  '11111111Q',
  '04444444T',
  '55675675J',
  datetime.timedelta(seconds=43200),
  datetime.timedelta(seconds=44100),
  datetime.timedelta(seconds=45900),
  datetime.timedelta(seconds=45)),
 ('0002',
  datetime.date(2020, 11, 11),
  16.5,
  '22222222S',
  '77777777M',
  '99009900J',
  datetime.timedelta(seconds=48600),
  datetime.timedelta(seconds=49500),
  datetime.timedelta(seconds=50700),
  datetime.timedelta(seconds=35)),
 ('0004',
  datetime.date(2020, 11, 10),
  14.3,
  '99999999X',
  '77777777M',
  '04477744T',
  datetime.timedelta(seconds=50520),
  datetime.timedelta(seconds=52200),
  datetime.timedelta(seconds=53100),
  datetime.timedelta(seconds=43)),
 ('0005',
  datetime.date(2020, 9, 5),
  14.0,
  '14567555L',
  '03232323P',
  '14188151T',
  datetime.timedelta(seconds=68520),
  datetime.timedelta(seconds=70200),
  datetime.timedelta(seconds=71100),
  datetime.timedelta(seconds=43)),
 ('0006',
  datetime.date(2020, 11, 15),
 

In [73]:
df23=pd.read_sql(query23_1, con=connection)
df23

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\583897665.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df23=pd.read_sql(query23_1, con=connection)


,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep,Tiempo_transcurrido
0,0001,2020-10-15,10.0,11111111Q,04444444T,55675675J,0 days 12:00:00,0 days 12:15:00,0 days 12:45:00,0 days 00:00:45
1,0002,2020-11-11,16.5,22222222S,77777777M,99009900J,0 days 13:30:00,0 days 13:45:00,0 days 14:05:00,0 days 00:00:35
2,0004,2020-11-10,14.3,99999999X,77777777M,04477744T,0 days 14:02:00,0 days 14:30:00,0 days 14:45:00,0 days 00:00:43
3,0005,2020-09-05,14.0,14567555L,03232323P,14188151T,0 days 19:02:00,0 days 19:30:00,0 days 19:45:00,0 days 00:00:43
4,0006,2020-11-15,25.3,14567555L,88888888O,04477744T,0 days 21:02:00,0 days 21:35:00,0 days 21:45:00,0 days 00:00:43
5,0007,2020-09-25,17.0,03232323P,88888888O,14188151T,0 days 23:05:00,0 days 23:12:00,0 days 23:35:00,0 days 00:00:30
6,0008,2020-09-15,14.0,99999999X,33333333M,11245621Q,0 days 18:02:00,0 days 18:30:00,0 days 18:45:00,0 days 00:00:43
7,0010,2020-11-05,49.5,88888888O,14567555L,11245621Q,0 days 22:05:00,0 days 22:12:00,0 days 22:20:45,0 days 00:00:15
8,0014,2021-01-15,13.0,99999999X,55555555J,04477744T,0 days 21:20:00,0 days 21:30:00,0 days 21:51:00,0 days 00:00:31


### Insertar en la tabla RANKING_PRODUCTOS por cada producto, su código, su nombre y la cantidad total pedida.(Debes hacerlo con una única sentencia).

In [74]:
query24=""" 
INSERT INTO 
	RANKING_PRODUCTOS (Codigo, Nombre, Total)
SELECT 
	pro.Codigo, pro.Nombre, SUM(con.Cantidad)
FROM 
	consta con
INNER JOIN 
	PRODUCTO pro ON con.Codigo_Pr = pro.Codigo
GROUP BY 
	pro.Codigo, pro.Nombre;
"""
cursor.execute(query24)
connection.commit()

In [75]:
query24_1=""" 
SELECT *
FROM
	RANKING_PRODUCTOS
"""
cursor.execute(query24_1)
cursor.fetchall()

[('02', 'Patatas', 2),
 ('04', 'Queso', 6),
 ('09', 'Coca cola', 1),
 ('11', 'Menú de Pollo', 4),
 ('12', 'Menú de Hamburguesa con queso', 11),
 ('13', 'Menú de Pollo con queso', 11),
 ('15', 'Helado', 2),
 ('16', 'Tarta', 3),
 ('18', 'Agua', 1),
 ('19', 'Fanta', 1)]

In [76]:
df24=pd.read_sql(query24_1, con=connection)
df24

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\3998631150.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df24=pd.read_sql(query24_1, con=connection)


,Codigo,Nombre,Total
0,02,Patatas,2
1,04,Queso,6
2,09,Coca cola,1
3,11,Menú de Pollo,4
4,12,Menú de Hamburguesa con queso,11
5,13,Menú de Pollo con queso,11
6,15,Helado,2
7,16,Tarta,3
8,18,Agua,1
9,19,Fanta,1


### Incrementar el salario de los empleados en 10 euros si han tomado nota de algún pedido. Además, esos empleados también deben haber preparado algún pedido. (Debes hacerlo con una única sentencia)

In [77]:
query24=""" 
UPDATE EMPLEADO
SET Salario = Salario + 10
WHERE DNI IN (
    SELECT DISTINCT DNI_ETM
    FROM PEDIDO
    WHERE DNI_ETM IS NOT NULL
    AND Numero IN (
        SELECT DISTINCT Numero_P
        FROM consta
    )
)
AND DNI IN (
    SELECT DISTINCT DNI_EP
    FROM PEDIDO
    WHERE DNI_EP IS NOT NULL
    AND Numero IN (
        SELECT DISTINCT Numero_P
        FROM consta
    )
);
"""
cursor.execute(query24)
connection.commit()

In [78]:
query25_1=""" 
SELECT *
FROM
	EMPLEADO
"""
cursor.execute(query25_1)
cursor.fetchall()

[('03232323P', 'María Luisa Galdón Ter', '43534534567', 'noche', 1210.0),
 ('04444444T', 'Úrsula Delta Camacho', '11177788899', 'mañana', 900.0),
 ('11111111Q', 'Luis Ramírez Pardo', '23445556666', 'mañana', 900.0),
 ('14111155T', 'Sergio Lérida Campos', '55577700089', 'tarde', 1000.0),
 ('14567555L', 'Piedad Colmenero Zapillo', '23456734534', 'noche', 1210.0),
 ('22222222S', 'María Sánchez Cid', '11112223334', 'tarde', 1000.0),
 ('33333333M', 'Martín Guerrero López', '33344455566', 'tarde', 1000.0),
 ('55555555J', 'Carmen Hernández Pío', '99966633311', 'mañana', 900.0),
 ('77777777M', 'Pedro Jiménez Ruiz', '23456785432', 'tarde', 1000.0),
 ('88888888O', 'Soledad Campillo Molina', '00088877754', 'noche', 1210.0),
 ('98567432x', 'Luis Fernández Cáceres', '09809809822', 'tarde', 850.0),
 ('99999999X', 'Raúl Rodrigo Roca', '55566677789', 'tarde', 1000.0)]

In [79]:
df25=pd.read_sql(query25_1, con=connection)
df25

C:\Users\Russhi\AppData\Local\Temp\ipykernel_23248\3796942633.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df25=pd.read_sql(query25_1, con=connection)


,DNI,Nombre,Nss,Turno,Salario
0,03232323P,María Luisa Galdón Ter,43534534567,noche,1210.0
1,04444444T,Úrsula Delta Camacho,11177788899,mañana,900.0
2,11111111Q,Luis Ramírez Pardo,23445556666,mañana,900.0
3,14111155T,Sergio Lérida Campos,55577700089,tarde,1000.0
4,14567555L,Piedad Colmenero Zapillo,23456734534,noche,1210.0
5,22222222S,María Sánchez Cid,11112223334,tarde,1000.0
6,33333333M,Martín Guerrero López,33344455566,tarde,1000.0
7,55555555J,Carmen Hernández Pío,99966633311,mañana,900.0
8,77777777M,Pedro Jiménez Ruiz,23456785432,tarde,1000.0
9,88888888O,Soledad Campillo Molina,00088877754,noche,1210.0


# Cierro la Conexión

In [80]:
cursor.close()
connection.close()

# Guardo la información en Excel

In [ ]:
with pd.ExcelWriter('SQLqueries_FastFoodCompany.xlsx',engine='xlsxwriter', mode='w') as writer:
    df1.to_excel(writer, sheet_name='Productos')
    df2.to_excel(writer, sheet_name='Repartidores')
    df3.to_excel(writer, sheet_name='SalarioEmpleados')
    df4.to_excel(writer, sheet_name='PedidoNov2020')
    df5.to_excel(writer, sheet_name='DNIRepartidores')
    df6.to_excel(writer, sheet_name='PedidosMes')
    df7.to_excel(writer, sheet_name='Empleados')
    df8.to_excel(writer, sheet_name='ProductosPrecioMedia')
    df9.to_excel(writer, sheet_name='EmpleadosNoPreparado')
    df10.to_excel(writer, sheet_name='PedidoLuisMaria')
    df11.to_excel(writer, sheet_name='TiemposRepartidor')
    df12.to_excel(writer, sheet_name='PedidosBaratoCaro')
    df13.to_excel(writer, sheet_name='Pedidos2')
    df14.to_excel(writer, sheet_name='IDEmpleado')
    df15.to_excel(writer, sheet_name='PedidoCodigo')
    df16.to_excel(writer, sheet_name='InsertoEmpleado')
    df17.to_excel(writer, sheet_name='Pedido010')
    df18.to_excel(writer, sheet_name='FrutaEliminado')
    df19.to_excel(writer, sheet_name='InsertarPedido')
    df20.to_excel(writer, sheet_name='Incremento10Rep')
    df21.to_excel(writer, sheet_name='RepartidorAlejandro')
    df22.to_excel(writer, sheet_name='Incentivo50')
    df23.to_excel(writer, sheet_name='Pedidos_Finalizados')
    df24.to_excel(writer, sheet_name='Ranking')
    df25.to_excel(writer, sheet_name='Incremento10Emple')
    
    writer.close()